# Домашнее задание 1. 

-  "Оптимальность" наивного Байесовского классификатора
- Оценка "важности" признаков (feature importance)

Для тех кто не знаком с библиотекой pandas я рекомендую вот эти два практических туториала:

-  https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb (1-3 часа на прохождение, 30 вопросов с видео разборами)
- https://github.com/guipsamora/pandas_exercises/ 11 топиков покрывающих весь основной функционал библиотеки


In [604]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

## 1. Наивный Байес

Ваша задача сравнить вероятности получаемые наивным байесовским классификатором с вероятностями получаемыми если не делать предположения о независимости признаков. Мы будем делать это для 2 признаков рассмотренных на семинаре: Пол и Класс билета (пассажира).

In [605]:
import pandas as pd

In [606]:
df = pd.read_csv('./titanic_data.csv')

In [607]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

## 1.1 Разделим данные на тренировочную и тестовую части 70-30

сколько наблюдений оказалось в тренировочной и тестовой выборках? В тренировочной 916, в тестовой 393

In [608]:
from sklearn.model_selection import train_test_split

In [609]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=5)

In [610]:
X_train.shape

(916, 2)

In [611]:
X_test.shape

(393, 2)

In [612]:
X_train.isnull().value_counts()

Sex    PassengerClass
False  False             916
dtype: int64

In [613]:
X_test.isnull().value_counts()

Sex    PassengerClass
False  False             393
dtype: int64

## 1.2 Реализуйте функцию `get_probability_survived_bayes`

Которая считает честные вероятности выживания при условии заданного класса билета и пола:

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{Passenger Class}, \text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}|\text{Passenger Class}) p(\text{Passenger Class})}
$$

## Помните что оценка вероятностей всегда должна происходить на тренировочной выборке.

In [614]:
from typing import Tuple

In [615]:
X_train["Sex"].value_counts()

0    597
1    319
Name: Sex, dtype: int64

In [616]:
X_train["PassengerClass"].value_counts()

3    493
1    243
2    180
Name: PassengerClass, dtype: int64

In [617]:
print(y_train.value_counts())
print(y_train.shape[0])

0    563
1    353
Name: Survived, dtype: int64
916


In [618]:
# Hint 1. Используйте pandas.groupby
# Hint 2. Вы возвращаете tuple p_not_survived, p_survived, в котором
# вероятность выжить находится на второй позиции, чтобы когда вы берете аргмакс получать метки классов 0/1 (не выжил/выжил)

def get_probability_survived_bayes(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    
    p_pass_class = X_train["PassengerClass"].value_counts()[pclass] / X_train["PassengerClass"].shape[0]
    p_sex_pass_class = X_train.groupby(['PassengerClass']).value_counts()[pclass][sex] / \
                        X_train.groupby(['PassengerClass']).value_counts()[pclass].sum()

    all = y_train.shape[0]
    surv = 1
    not_surv = 0
    p_not_survived_prior = y_train.value_counts()[not_surv] / all
    p_survived_prior = y_train.value_counts()[surv] / all

    X_full = X_train.join(y_train)
    p_pass_class_not_survived = X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][not_surv][pclass] / \
                                X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][not_surv].sum()
    
    p_pass_class_survived = X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][surv][pclass] / \
                                X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][surv].sum()
    
    p_sex_pass_class_not_survived = X_full.groupby(["PassengerClass", "Survived"]).value_counts().to_frame()[0][pclass][not_surv][sex] / \
                                X_full.groupby(["PassengerClass", "Survived"]).value_counts().to_frame()[0][pclass][not_surv].sum()
    
    p_sex_pass_class_survived = X_full.groupby(["PassengerClass", "Survived"]).value_counts().to_frame()[0][pclass][surv][sex] / \
                                X_full.groupby(["PassengerClass", "Survived"]).value_counts().to_frame()[0][pclass][surv].sum()
    
    p_not_survived = (p_sex_pass_class_not_survived * p_pass_class_not_survived * p_not_survived_prior) / \
                        (p_sex_pass_class * p_pass_class)
    p_survived = (p_sex_pass_class_survived * p_pass_class_survived * p_survived_prior) / \
                        (p_sex_pass_class * p_pass_class)
    return(p_not_survived, p_survived)

In [619]:
get_probability_survived_bayes(1,0)

(0.6417910447761194, 0.3582089552238805)

## 1.3 Реализуйте функцию `get_probability_survived_bayes_naive`

Которая оценивает вероятность выжить в предположении о независимости признаков

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}) p(\text{Passenger Class})}
$$

In [620]:
def get_probability_survived_bayes_naive(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns tuple(p_not_survived, p_survived), 
    under naive assumption of features independence'''
    ...
    
# Hint 1. Поскольку вы делаете предположении о независимости признаков, то p(y=1|...) + p(y=0| ...) некорректно отнормированы
# и не будут суммироваться в 1. Не забудьте выполнить необходимую нормировку.
    
    p_pass_class = X_train["PassengerClass"].value_counts()[pclass] / X_train["PassengerClass"].shape[0]
    p_sex = X_train["Sex"].value_counts()[sex] / X_train["Sex"].shape[0]

    all = y_train.shape[0]
    surv = 1
    not_surv = 0
    p_not_survived_prior = y_train.value_counts()[not_surv] / all
    p_survived_prior = y_train.value_counts()[surv] / all

    X_full = X_train.join(y_train)
    p_pass_class_not_survived = X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][not_surv][pclass] / \
                                X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][not_surv].sum()
    
    p_pass_class_survived = X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][surv][pclass] / \
                                X_full.groupby(["Survived", "PassengerClass"]).count()['Sex'][surv].sum()
    
    p_sex_not_survived = X_full.groupby(["Survived", "Sex"]).count()["PassengerClass"][not_surv][sex] / \
                                X_full.groupby(["Survived", "Sex"]).count()["PassengerClass"][not_surv].sum()
        
    p_sex_survived = X_full.groupby(["Survived", "Sex"]).count()["PassengerClass"][surv][sex] / \
                                X_full.groupby(["Survived", "Sex"]).count()["PassengerClass"][surv].sum()
    
    p_not_survived = (p_sex_not_survived * p_pass_class_not_survived * p_not_survived_prior) / \
                    (p_sex * p_pass_class)
    p_survived = (p_sex_survived * p_pass_class_survived * p_survived_prior) / \
                    (p_sex * p_pass_class)
    norm_p_not_survived = p_not_survived / (p_not_survived + p_survived)
    norm_p_survived = p_survived / (p_not_survived + p_survived)
    return (norm_p_not_survived, norm_p_survived)

In [621]:
get_probability_survived_bayes_naive(1,0)

(0.61236332199898, 0.38763667800102003)

## 1.4 Используйте ваши функции для того чтобы сделать предсказание на тесте


Для всех 6 уникальных комбинаций значений признаков сравните вероятности принадлежности классу 1
 - насколько они отличаются?
 Ответ: отличия совсем незначительные
 - влияют ли эти отличия на предсказание класса объекта, при пороге $p = 0.5$?
 Ответ: не влияют

In [622]:
from tqdm.notebook import tqdm

In [623]:
y_naive_proba_dict = {}

for _, (sex, pclass) in X_test.iterrows():
    y_naive_proba_dict[(sex, pclass)] = get_probability_survived_bayes_naive(pclass, sex)

In [624]:
y_proba_dict = {}

for _, (sex, pclass) in X_test.iterrows():
    y_proba_dict[(sex, pclass)] = get_probability_survived_bayes(pclass, sex)

In [625]:
y_naive_proba_dict

{(0, 2): (0.8047975487446595, 0.19520245125534053),
 (1, 2): (0.24061063532344093, 0.759389364676559),
 (0, 3): (0.8834204092763698, 0.11657959072363035),
 (1, 3): (0.3680337886313606, 0.6319662113686394),
 (0, 1): (0.61236332199898, 0.38763667800102003),
 (1, 1): (0.10826076998031765, 0.8917392300196825)}

In [626]:
import numpy as np
for i in y_naive_proba_dict:
    print(i, np.argmax(y_naive_proba_dict[i]))

(0, 2) 0
(1, 2) 1
(0, 3) 0
(1, 3) 1
(0, 1) 0
(1, 1) 1


In [627]:
y_proba_dict

{(0, 2): (0.8771929824561405, 0.12280701754385966),
 (1, 2): (0.13636363636363638, 0.8636363636363639),
 (0, 3): (0.8538681948424067, 0.1461318051575931),
 (1, 3): (0.4583333333333333, 0.5416666666666666),
 (0, 1): (0.6417910447761194, 0.3582089552238805),
 (1, 1): (0.03669724770642202, 0.9633027522935779)}

In [628]:
for i in y_proba_dict:
    print(i, np.argmax(y_naive_proba_dict[i]))

(0, 2) 0
(1, 2) 1
(0, 3) 0
(1, 3) 1
(0, 1) 0
(1, 1) 1


## 1.5 Обучите имплементацию `MultinomialNB` из `sklearn`

Тот же вопрос что и 1.4, прокомментируйте отличия в оценке вероятности ваших имлементаций и имплементации
`MultinomialNB`. Влияют ли они на предсказание классов при стандартном пороге вероятности (p=0.5)? Заполните таблицу `probabilities`.

In [629]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

clf.fit(X_train, y_train)

features = pd.DataFrame({'Sex':[0,0,0,1,1,1], 'PassengerClass':[1,2,3,1,2,3]})
arr = clf.predict_proba(features)

y_pred_2 = clf.predict(X_test)

In [630]:
probabilities = pd.DataFrame()
probabilities['Sex'] = [0,0,0,1,1,1]
probabilities['Passenger Class'] = [1,2,3,1,2,3]
probabilities['Bayes'] = [y_proba_dict[(probabilities['Sex'][i],probabilities['Passenger Class'][i])][1] for i in range(6)]
probabilities['BayesNaive'] = [y_naive_proba_dict[(probabilities['Sex'][i],probabilities['Passenger Class'][i])][1] for i in range(6)]
probabilities['MultinomialNB'] = [arr[i][1] for i in range(6)]

probabilities

,Sex,Passenger Class,Bayes,BayesNaive,MultinomialNB
0,0,1,0.358209,0.387637,0.328826
1,0,2,0.122807,0.195202,0.276841
2,0,3,0.146132,0.116580,0.230255
3,1,1,0.963303,0.891739,0.702877
4,1,2,0.863636,0.759389,0.648932
5,1,3,0.541667,0.631966,0.590894


## 2.1 Используя MultinomialNB обучите классификатор на бОльшем количестве признаков

1. Подготовьте признаки 
- 'Passenger Class',
- 'Sex',
- 'Age',
- 'No of Siblings or Spouses on Board',
- 'No of Parents or Children on Board',
- 'Port of Embarkation'

для обучения:
- закодируйте столбец `Port of Embarkation` по аналогии с `Passenger Class`
- заполните пропуски в столбце `Age` средним возрастом в датасете используйте `pd.DataFrame.fillna(...)`
- заполните пропуске в столбце `Port of Embarkation` самым частым портом посадки
- для разбиения на тренировчную и тестовую часть данных используйте `train_size=0.66` и `random_state=5`

2. обучите MultinomialNB модель на тренировочной части данных
2. Как изменилась точность вашей модели на тестовой части данных по сравнению с моделью обученной на 2 признаках?
    Ответ: точность увеличилась на одну сотую
3. Что означает признак `Life Boat`? Следуют ли на ваш взгляд включать его в список признаков для обучения модели? 
    Ответ: признак 'Life Boat' означает в какую спасательную лодку был посажен пассажир. По моему мнению, включать его в список признаков не имеет смысла, так как в нем 823 пропущенных значений.
- Опишите сценарий (использования модели) в котором его следует включать в список признаков. 
Ответ: с другой стороны он дает очень высокую процентную выживаемость, так как если лодка доплыла, то с высокой вероятностью человек выжил, то есть если просто есть номер лодки, то уже можно сказать, что человек выжил
- Опишите сценарий (использования модели) в котором его не следует включать в список признаков.
Ответ: но в большинстве случаев нет информации о лодке, хотя человек выжил, что приведет в неправильной работе модели
5. Какая ошибка содержится в предложенной последовательности действий в п. 2.1? 
    Ответ: возможно, сначала необходимо заполнить пропуски в столбце `Port of Embarkation` самым частым портом посадки, а после уже закодировать его. Кроме того, возможно не стоило заменять пропущенные значения средним значением.

In [631]:
df.head()

,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat,Survived
0,First,"Allen, Miss. Elisabeth Walton",Female,29.0000,0,0,24160,211.3375,B5,Southampton,2,Yes
1,First,"Allison, Master. Hudson Trevor",Male,0.9167,1,2,113781,151.5500,C22 C26,Southampton,11,Yes
2,First,"Allison, Miss. Helen Loraine",Female,2.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
3,First,"Allison, Mr. Hudson Joshua Creighton",Male,30.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No
4,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25.0000,1,2,113781,151.5500,C22 C26,Southampton,NaN,No


In [632]:
# df['Port of Embarkation'].value_counts()
# df['Age'].isna().sum()
# df['Life Boat'].value_counts()
df['Life Boat'].isna().sum()

823

In [633]:
X_n = pd.DataFrame()

X_n['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X_n['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X_n['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y_n = X_n['Survived']
X_n = X_n.drop(['Survived'], axis=1)

X_n['Port'] = df['Port of Embarkation'].map({
    'Southampton': 1,
    'Cherbourg': 2,
    'Queenstown': 3
})
X_n['Age'] = df['Age'].fillna(df['Age'].mean())
X_n['Port'] = X_n['Port'].fillna('1')
X_n['NoOfSiblings'] = df['No of Siblings or Spouses on Board']
X_n['NoParents'] = df['No of Parents or Children on Board']

In [634]:
X_n.isna().sum()

Sex               0
PassengerClass    0
Port              0
Age               0
NoOfSiblings      0
NoParents         0
dtype: int64

In [635]:
X_n.head()

,Sex,PassengerClass,Port,Age,NoOfSiblings,NoParents
0,1,1,1.0,29.0000,0,0
1,0,1,1.0,0.9167,1,2
2,1,1,1.0,2.0000,1,2
3,0,1,1.0,30.0000,1,2
4,1,1,1.0,25.0000,1,2


In [636]:
X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, y_n, train_size=0.66, random_state=5)

In [637]:
model = MultinomialNB()
model.fit(X_n_train, y_n_train)

MultinomialNB()

In [638]:
y_pred = model.predict(X_n_test)
print('6 features', accuracy_score(y_n_test, y_pred))
print('2 features', accuracy_score(y_test,y_pred_2))

6 features 0.7668161434977578
2 features 0.7557251908396947


## 2.2. Оценка важности признаков

Мы воспользуемся простым (и не очень хорошим^[1]) методом оценки важности признаков:

1. Воспользуйтесь моделью из п.2. Качество модели на тестовой части данных это ваш бейзлайн
2. Исключая по 1 признаку из `X` обучайте модель `MultinomialNB` и измеряйте ее точность на тестовой части данных
3. Сохраняйте величину падения или роста качества по сравнению с бейзлайном. Если точность упала значит признак "полезный", если выросло - "вредный". Абсолютная величина падения или роста в данном случае не так важна, важен только порядок признаков. Такая оценка называется качественной (в противовес количественной).
4. Прокомментируйте полученный результат, насколько он согласуется с вашей интуицией?
Ответ: самая большая корреляция связана с полом, что совпадает с интуцией. На удивление, отрицательная корреляция связана с возрастом, что скорее всего связано с тем, что было много пропущенных значений, которые мы заполнили средним значением.
5. В чем на ваш взляд могут быть преимущества использования мЕньшего числа признаков при обучении моделей? (не обязательно в контексте данной задачи)
Ответ: не будет переобучения, ускорение процесса

[1] а впрочем какие из них хороши?

In [639]:
baseline = accuracy_score(y_n_test, y_pred)
X_n.columns[0]

'Sex'

In [640]:
drop = {}
for i in X_n.columns:
    model.fit(X_n_train.drop([i], axis=1), y_n_train)
    y_pred_1 = model.predict(X_n_test.drop([i], axis=1))
    acc = accuracy_score(y_n_test, y_pred_1)
    drop[i]= (acc - baseline)

In [641]:
drop

{'Sex': -0.10986547085201792,
 'PassengerClass': -0.022421524663677084,
 'Port': -0.004484304932735439,
 'Age': 0.0022421524663677195,
 'NoOfSiblings': 0.0,
 'NoParents': -0.0022421524663677195}

jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма `[ml-ds-2024-ha1-Surname]`, например `[ml-ds-2023-Kurmukov]` до 12:00 МСК 08.02.2023. Задания отправленные до полуночи Вторника 06.02.2023 получают +1 балл в случае полного выполнения (не обязательно верного:))